In [34]:
filename = "sample"
original_path = filename + ".txt"
compressed_path = filename + "_LZW_compressed.bit"
recon_path = filename + "_LZW_recon.txt"

In [35]:
def bin2utf8(string):
    message = ""
    while string != "":
        i = chr(int(string[:8], 2))
        message = message + i
        string = string[8:]
    return(message)

In [38]:
# LZW method for compression and decompression

import numpy as np

Document = open(original_path, "r").read()
Dictionary, EncoderOutput, = {}, np.array([])


def InitializeDict(file):
    for char in file:
        if char not in Dictionary.keys():
            Dictionary[char] = len(Dictionary)

    np.save("InitializedDictionaryKeys.npy", np.array(
        list(Dictionary.keys())), allow_pickle=True)
    np.save("InitializedDictionaryValues.npy", np.array(
        list(Dictionary.values())), allow_pickle=True)


def compress_LZW(file):
    global Dictionary
    global EncoderOutput

    index = 0

    while index < len(file):
        TempIndex, TrackingString = index+1, file[index]

        while TempIndex < len(file) and (TrackingString + file[TempIndex]) in Dictionary.keys():
            TrackingString += file[TempIndex]
            TempIndex += 1

        EncoderOutput = np.append(EncoderOutput, Dictionary[TrackingString]))
        EncoderOutput = EncoderOutput.astype(int) 
        if TempIndex < len(file):
            Dictionary[TrackingString+file[TempIndex]] = len(Dictionary)

        index = TempIndex

    np.save(filename + "_LZW_compressed.npy", EncoderOutput, allow_pickle=True)
    print("encoder output: ",EncoderOutput)
    print("encoder output type: ", EncoderOutput.dtype)
    compressed = bin2utf8(EncoderOutput)
    print("compressed:", compressed)
    with open(compressed_path,"wb") as compressed_file:
        compressed_file.write(compressed)

In [39]:
%%time
if __name__ == "__main__":
    InitializeDict(Document)
    compress_LZW(Document)

encoder output:  [0 1 0 2 1 3 9 2 0 0 3 4]
encoder output type:  int32


<ipython-input-35-25accf25e5b8>:3: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  while string != "":


TypeError: int() can't convert non-string with explicit base

In [6]:
import numpy as np

EncoderOutput = np.array([])
Document, Dictionary = "", {}


def Load():
    global EncoderOutput
    
    EncoderOutput = np.load(filename + "_LZW_compressed.npy", allow_pickle=True)
    keys, values = np.load("InitializedDictionaryKeys.npy", allow_pickle=True), np.load(
        "InitializedDictionaryValues.npy", allow_pickle=True)

    for i in range(len(keys)):
        Dictionary[values[i]] = keys[i]


def recon_LZW():
    global EncoderOutput
    global Dictionary
    global Document

    Document += Dictionary[EncoderOutput[0]]
    TempString = Dictionary[EncoderOutput[0]]

    for index in range(1, len(EncoderOutput)):
        Document += Dictionary[EncoderOutput[index]]
        TempString += Dictionary[EncoderOutput[index]]
        TempIndex = 1

        while TempIndex <= len(TempString) and TempString[:TempIndex] in Dictionary.values():
            TempIndex += 1

        if TempString[:TempIndex] not in Dictionary.values():
            Dictionary[len(Dictionary)] = TempString[:TempIndex]
            TempString = TempString[TempIndex-1:]

    open(recon_path, "w").write(Document)

In [7]:
%%time
if __name__ == "__main__":
    Load()
    recon_LZW()

Wall time: 5 ms


In [8]:
import os
def compareFile(path1 = "sample.txt",path2 = "sample_LZW_recon.txt"):
    if os.stat(path1).st_size ==0 or os.stat(path2).st_size == 0:
        print("\tFile is empty")
        return
    f1 = open(path1, "r")  
    f2 = open(path2, "r")   
    i = 0

    for line1 in f1:
        i += 1      
        for line2 in f2:
            if line1 != line2:
                print("Differs at line", i, ":")
                # else print that line from both files
                print("\tFile 1:", line1, end='')
                print("\tFile 2:", line2, end='')
                return
            break
    print("The input file and reconstructed file have no difference")
    # closing files
    f1.close()
    f2.close()

In [9]:
print("LZW compression: ")
print("Compressed file path: " + compressed_path)
print("Decompressed file path: " + recon_path)

length_original = os.path.getsize(original_path) 
length_compressed = os.path.getsize(compressed_path) 
compareFile(original_path, recon_path)
print("size of original text is:",length_original)
print("size of compressed text is:",length_compressed)
print("rate of compression is:",length_original/length_compressed)

LZW compression: 
Compressed file path: sample_LZW_compressed.bit
Decompressed file path: sample_LZW_recon.txt
The input file and reconstructed file have no difference
size of original text is: 10
size of compressed text is: 40
rate of compression is: 0.25
